In [1]:
from torchvision import datasets
import torch
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_folder = './content'
fmnist = datasets.FashionMNIST(data_folder, download=True, train=True)

tr_images = fmnist.data
tr_targets = fmnist.targets
val_fmnist = datasets.FashionMNIST(data_folder,
                                   download=True,
                                   train=False)

val_images = val_fmnist.data
val_targets = val_fmnist.targets

In [2]:
device

device(type='cuda')

In [3]:
#%pip install imgaug

import albumentations as A

aug = A.Compose([
    A.Affine(translate_px={'x': (-10, 10), 'y': (0, 0)}, mode=0, p=1.0)
])

C:\Users\guilh\AppData\Local\Temp\ipykernel_9856\141358394.py:6: UserWarning: Argument(s) 'mode' are not valid for transform Affine
  A.Affine(translate_px={'x': (-10, 10), 'y': (0, 0)}, mode=0, p=1.0)


In [22]:
class FMNISTDataset(Dataset):
    def __init__(self, x, y, aug=None):
        self.x, self.y = x, y
        self.aug = aug

    def __getitem__(self, ix):
        x, y = self.x[ix], self.y[ix]
        return x, y

    def __len__(self):
        return len(self.x)

    def collate_fn(self, batch):
        ims, classes = list(zip(*batch))
        ims_np = np.stack([im.numpy() for im in ims])
        if self.aug:    
            ims_aug = []
            for im in ims_np:
                im = np.expand_dims(im, axis=-1)
                augmented = self.aug(image=im)
                ims_aug.append(augmented['image'])
            ims_np = np.stack(ims_aug)
            ims_np = ims_np.squeeze(-1)
        ims = torch.from_numpy(ims_np)[:, None, :, :].float() / 255.0
        classes = torch.tensor(classes)
        return ims, classes


In [ ]:
from torch.optim import AdamW

# A grande alteração aqui foi na arquitetura do modelo, que agora inclui mais camadas convolucionais e uma camada de dropout.
# O desempenho mudou levemente, com cerca de 2-4% de melhoria na acurácia de validação.
def get_model():
    model = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=3, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Conv2d(32, 64, kernel_size=3, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2),
        
        nn.Conv2d(64, 128, kernel_size=3, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Flatten(),
        nn.Linear(128 * 3 * 3, 256),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(256, 10)
    ).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=1e-3)
    return model, loss_fn, optimizer

In [ ]:
def train_batch(x, y, model, opt, loss_fn):
    model.train()
    prediction = model(x)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    opt.step()
    opt.zero_grad()
    return batch_loss.item()

In [25]:
def get_data():
    train = FMNISTDataset(tr_images, tr_targets, aug=aug)  
    # Observe o argumento collate_fn
    trn_dl = DataLoader(train, batch_size=64, 
                        collate_fn=train.collate_fn, shuffle=True)
    val = FMNISTDataset(val_images, val_targets)
    val_dl = DataLoader(val, batch_size=len(val_images),
                        collate_fn=val.collate_fn, shuffle=True)
    return trn_dl, val_dl

In [31]:
trn_dl, val_dl = get_data()
model, loss_fn, optimizer = get_model()
for epoch in range(5):
    print(f"Epoch {epoch}")  # Exibe a época atual
    for ix, batch in enumerate(iter(trn_dl)):
        x, y = batch  # Obtém o lote de dados
        x = x.to(device)
        y = y.to(device)
        batch_loss = train_batch(x, y, model, optimizer, loss_fn)  # Treina o modelo e calcula a perda

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4


In [32]:
# Avaliando o modelo
model.eval()
with torch.no_grad():
    for ix, batch in enumerate(val_dl):
        x, y = batch 
        x = x.to(device)
        y = y.to(device)
        prediction = model(x)  # Faz a previsão
        val_loss = loss_fn(prediction, y)  # Calcula a perda de validação
        print(f"Validation Loss: {val_loss.item()}")  # Exibe a perda de validação

# Analisando a acurácia do modelo
correct = 0
total = 0
with torch.no_grad():
    for ix, batch in enumerate(val_dl):
        x, y = batch
        x = x.to(device)
        y = y.to(device)
        prediction = model(x)  # Faz a previsão
        _, predicted = torch.max(prediction.data, 1)  # Obtém as classes previstas
        total += y.size(0)  # Total de amostras
        correct += (predicted == y).sum().item()  # Contagem de acertos
accuracy = 100 * correct / total
print(f'Acurácia do modelo nos dados de validação: {accuracy:.2f}%')

Validation Loss: 0.33015939593315125
Acurácia do modelo nos dados de validação: 87.85%
